# Dataset and Data Wrangling:
The data used in this study includes natality (birth data) and fatality (death data) files of infants born in the year 2008 accompanied by a pdf-formatted document that describes the data content and type. The birth cohort data for 2008 consists of infant deaths that occurred in 2008 or 2009 linked to births in 2008. The data also includes a separate file that includes infant deaths, unlinked file, which had not been linked to a corresponding record in the natality file. 

The guide document was converted into usable format in two steps. 

First, Tabula, service software, was used to convert the guide document from pdf to tsv (tab separated values file) format. The tsv file was then reformatted using python

The original data set did not have column names and it required a second step of writing a python code to extract the field names from the accompanying guide document. 

An exploratory data assessment shows the number of fields and type of some of the fields has changed over the years. Although the original plan was to work with data from 1995 to 2010, I made the decision of working with the 2008 data 
(4 million plus records) for this project.

In [1]:
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
%matplotlib inline

The following function extracts only the necessary attribute information (the name of the attribute and its poistion in the data) from the guide file and reformats it for further analysis.

In [2]:
import codecs
import re
def format_cols():
    cols_in = codecs.open('2005/tabula_2005','r','utf-8')
    file_out=open('2005/format_out','w+')
    for lines in cols_in:
        try:
            cols = lines.split()
            if (re.match("^[0-9]",cols[0]) and (re.match("FILLER",cols[2]) is None) and (re.match("[A-Z][A-Z]",cols[2]) is not None)):
                file_out.write(cols[0] + '\t' + cols[1] + '\t' + cols[2] + '\n')
            elif (re.match('\"',cols[0]) and (re.match("^[0-9]",cols[1])) and (re.match("^[0-9]",cols[2])) and (re.match("[A-Z][A-Z]",cols[3]) is not None)):
                file_out.write(cols[1] + '\t' + cols[2] + '\t' + cols[3] + '\n')
        except:
            pass

#format_cols()

The following function extracts the position of each attribute from the reformatted and cleaned data guide file

In [2]:
data_yearList = [2008]

In [15]:
import re
def instfile():
    global data_yearList
    for yr in data_yearList:
        path_name = os.path.join('ReadingData/',str(yr),'new_format')
        infile = open(path_name,'r')
        outfile = open('%s.out.txt' % yr,'w+')
        outfile2 = open('%s.out2.txt' % yr,'w+')
        ou3 = open('%s.colname' % yr,'w+')
   
        for lines in infile:
            c1 = re.split('\t',lines)[0].replace("-",",")
            x = int(re.split(',',c1)[0]) - 1 
            x = str(x)
            try:
                y = re.split(',',c1)[1]
            except IndexError:
                y = int(re.split(',',c1)[0]) 
                y = str(y)
                
            #print(x,y)
            c2 = re.split('\t',lines)[2]
            outfile.write(x + "\n")
            outfile2.write(y + "\n")
            ou3.write(c2 + "\n")
    return x,y
    #return c1, c2
#[c1,c2]=instfile()
[x,y]=instfile()


The following function builds a data frame using the attribute information (from the guide file) and the actual data. 
It has the capability of building a dictionary for multiple years if needed.

In [3]:
#sam_year = [1995,1996]
def build_dict(yr):
    fname = str(yr) + '/LinkCO' + str(yr)[2:4] + 'USDen.dat'
    #path_name = os.path.abspath('ReadingData/' + str(yr) + fname)
    path_name = os.path.join('ReadingData/',fname)
    print(path_name)
    infl = pd.read_table(path_name,'r',header=None)
    ofl1=pd.read_csv('%s.out.txt' % yr,'r',header=None)
    ofl2=pd.read_csv('%s.out2.txt' % yr,'r',header=None)
    ofl3=pd.read_csv('%s.colname' % yr,'r',header=None)
    a_name = 'a_{}'.format(yr) 
    a_name = {}
    for ind in range(len(ofl3.index)):
        a_name[ofl3[0][ind]]=(infl.iloc[:,0].str.slice(ofl1.iat[ind,0],ofl2.iat[ind,0]))
    df = pd.DataFrame(a_name)
    return df

yrList = [2008]
def dict_all(yrList):
    d={}
    for yr in yrList:
        d['%s' % yr] = build_dict(yr)
    return d
dd = dict_all(yrList)

ReadingData/2008/LinkCO08USDen.dat


The data is now ready for the next step of exploratory data analysis (EDA). We may come back to the data wrangling process if we decide any reformatting makes the data analysis part easier


In [9]:
def num_missing(col_or_row):
    return sum(col_or_row.isnull())
print(aa.apply(num_missing, axis=0))

AB_ANTI       0
AB_AVEN1      0
AB_AVEN6      0
AB_BINJ       0
AB_NICU       0
AB_SEIZ       0
AB_SURF       0
AGED          0
AGER22        0
AGER5         0
APGAR5        0
APGAR5R       0
ATTEND        0
AUTOPSY       0
BFACIL        0
BFACIL3       0
BRTHWGT       0
BWTIMP        0
BWTR14        0
BWTR4         0
CA_ANEN       0
CA_CCHD       0
CA_CDH        0
CA_CLEFT      0
CA_CLPAL      0
CA_DISOR      0
CA_DOWN       0
CA_GAST       0
CA_HYPO       0
CA_LIMB       0
             ..
UCA_DOWNS     0
UCA_OMPHA     0
UCA_SPINA     0
UCIG_REC6     0
UCODR130      0
UDMETH_REC    0
UFAGECOMB     0
UFHISP        0
UFRACE        0
ULD_BREECH    0
ULD_MECO      0
ULD_PRECIP    0
UMEDUC        0
UME_FORCP     0
UME_PRIMC     0
UME_REPEC     0
UME_VAC       0
UME_VAG       0
UME_VBAC      0
UMHISP        0
UOP_INDUC     0
UOP_TOCOL     0
UPREVIS       0
URF_CHYPER    0
URF_DIAB      0
URF_ECLAM     0
URF_PHYPER    0
WEEKDAYD      0
WTGAIN        0
WTGAIN_REC    0
dtype: int64


In [5]:
import matplotlib.pyplot as plt
 
# Data to plot
labels = '1', '2', '3', '4', '5'
sizes = [23713, 49885, 480315, 3433278, 19852]
colors = ['red','gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
explode = (0.1, 0, 0, 0,0)  # explode 1st slice
 
# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=140)
 
plt.axis('equal')
plt.show()

In [2]:
import matplotlib.pyplot as plt
 
labels = ['1', '2', '3', '4', '5']
data1 = [23713, 49885, 480315, 3433278, 19852]
data2= [9705, 3108, 4151, 6435, 774]
colors = ['red','gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
fig, (ax1, ax2) = plt.subplots(1, 2)
patches, texts = ax1.pie(data1, colors=colors, shadow=True, startangle=90)
patches, texts = ax2.pie(data2, colors=colors, shadow=True, startangle=90)
plt.legend(patches, labels, loc="best")
plt.axis('equal')
plt.tight_layout()
plt.show()

/Users/sbisrat/anaconda/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
